In [1]:
from qdrant_client import QdrantClient, models

qd_client = QdrantClient(url="http://localhost:6333")

In [5]:
qd_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='zoomcamp-faq')])

In [4]:
qd_client.delete_collection(collection_name="zoomcamp-sparse")

True

In [6]:
qd_client.create_collection(
    collection_name="zoomcamp-sparse",
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [7]:
import uuid

# Send the points to the collection
qd_client.upsert(
    collection_name="zoomcamp-sparse",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            }
        )
        for course in documents_raw
        for doc in course["documents"]
    ]
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

dutch.txt:   0%|          | 0.00/453 [00:00<?, ?B/s]

danish.txt:   0%|          | 0.00/424 [00:00<?, ?B/s]

arabic.txt: 0.00B [00:00, ?B/s]

...

italian.txt: 0.00B [00:00, ?B/s]

spanish.txt: 0.00B [00:00, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
# Step 3: Running sparse vector search with BM25

def search(query: str, limit: int=1)->list[models.ScoredPoint]:
    results = qd_client.query_points(
        collection_name="zoomcamp-sparse",
        query=models.Document(
            text=query,
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [9]:
results = search("qdrant")
results

[]

In [10]:
results = search("pandas")
results

[ScoredPoint(id='9127a578-d358-4dd1-bd31-e5049926df84', version=0, score=6.0392046, payload={'text': "You can use round() function or f-strings\nround(number, 4)  - this will round number up to 4 decimal places\nprint(f'Average mark for the Homework is {avg:.3f}') - using F string\nAlso there is pandas.Series. round idf you need to round values in the whole Series\nPlease check the documentation\nhttps://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.round.html#pandas.Series.round\nAdded by Olga Rudakova", 'section': '2. Machine Learning for Regression', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)]

In [12]:
print(results[0].payload["text"])

You can use round() function or f-strings
round(number, 4)  - this will round number up to 4 decimal places
print(f'Average mark for the Homework is {avg:.3f}') - using F string
Also there is pandas.Series. round idf you need to round values in the whole Series
Please check the documentation
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.round.html#pandas.Series.round
Added by Olga Rudakova


In [13]:
results[0].score

6.0392046

In [16]:
# Natural language like queries
import random
import json

random.seed(202506)

course = random.choice(documents_raw)
course_piece = random.choice(course["documents"])

print(json.dumps(course_piece, indent=2))

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}


In [17]:
results = search(course_piece["question"])
print(results[0].payload["text"])

The trial dbt account provides access to dbt API. Job will still be needed to be added manually. Airflow will run the job using a python operator calling the API. You will need to provide api key, job id, etc. (be careful not committing it to Github).
Detailed explanation here: https://docs.getdbt.com/blog/dbt-airflow-spiritual-alignment
Source code example here: https://github.com/sungchun12/airflow-toolkit/blob/95d40ac76122de337e1b1cdc8eed35ba1c3051ed/dags/examples/dbt_cloud_example.py


# Step 4: Qdrant universal Query API - prefetching

allows building multi-step search pipelines which can incorporate various methods into a single call.
For example, we can retrieve some candidates with dense vector search, and then rerank them with sparse search,
or use a fast method for initial retrieval and precise, but slow, reranking

In [22]:
qd_client.create_collection(
    collection_name="zoomcamp-sparse-and-dense",
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        "jina-small": models.VectorParams(
            size=512,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [23]:

# Send the points to the collection
qd_client.upsert(
    collection_name="zoomcamp-sparse-and-dense",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=doc["text"], 
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            }
        )
        for course in documents_raw
        for doc in course["documents"]
    ]
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
def multi_stage_search(query: str, limit: int=1)->list[models.ScoredPoint]:
    results = qd_client.query_points(
        collection_name="zoomcamp-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=(10*limit),
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [34]:
results = multi_stage_search(course_piece["question"])
print(results[0].payload["text"])

Problem description. How can we connect s3 bucket to MLFLOW?
Solution: Use boto3 and AWS CLI to store access keys. The access keys are what will be used by boto3 (AWS' Python API tool) to connect with the AWS servers. If there are no Access Keys how can they make sure that they have the right to access this Bucket? Maybe you're a malicious actor (Hacker for ex). The keys must be present for boto3 to talk to the AWS servers and they will provide access to the Bucket if you possess the right permissions. You can always set the Bucket as public so anyone can access it, now you don't need access keys because AWS won't care.
Read more here: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html
Added by Akshit Miglani


In [35]:
results

[ScoredPoint(id='733867b9-9ad0-444b-9d63-4933a378a7e0', version=0, score=28.892433, payload={'text': "Problem description. How can we connect s3 bucket to MLFLOW?\nSolution: Use boto3 and AWS CLI to store access keys. The access keys are what will be used by boto3 (AWS' Python API tool) to connect with the AWS servers. If there are no Access Keys how can they make sure that they have the right to access this Bucket? Maybe you're a malicious actor (Hacker for ex). The keys must be present for boto3 to talk to the AWS servers and they will provide access to the Bucket if you possess the right permissions. You can always set the Bucket as public so anyone can access it, now you don't need access keys because AWS won't care.\nRead more here: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html\nAdded by Akshit Miglani", 'section': 'Module 4: Deployment', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)]

In [26]:
def rrf_search(query: str, limit: int=1)->list[models.ScoredPoint]:
    results = qd_client.query_points(
        collection_name="zoomcamp-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=(5*limit),
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="bm25",
                limit=(5*limit),
            ),
        ],

        #Fusion query enables fusion on thye prefetched results
        query=models.FusionQuery(
            fusion=models.Fusion.RRF,
        ),
        with_payload=True,
    )

    return results.points

In [32]:
results = rrf_search(course_piece["question"])
print(json.dumps(course_piece, indent=2))
print(results[0].payload["text"])

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}
When executing an AWS CLI command (e.g., aws s3 ls), you can get the error <botocore.awsrequest.AWSRequest object at 0x7fbaf2666280>.
To fix it, simply set the AWS CLI environment variables:
export AWS_DEFAULT_REGION=eu-west-1
export AWS_ACCESS_KEY_ID=foobar
export AWS_SECRET_ACCESS_KEY=foobar
Their value is not important; anything would be ok.
Added by Giovanni Pecoraro


In [33]:
results

[ScoredPoint(id='cc25385d-ff06-4120-aa66-8db1c1b687c0', version=0, score=0.5, payload={'text': 'When executing an AWS CLI command (e.g., aws s3 ls), you can get the error <botocore.awsrequest.AWSRequest object at 0x7fbaf2666280>.\nTo fix it, simply set the AWS CLI environment variables:\nexport AWS_DEFAULT_REGION=eu-west-1\nexport AWS_ACCESS_KEY_ID=foobar\nexport AWS_SECRET_ACCESS_KEY=foobar\nTheir value is not important; anything would be ok.\nAdded by Giovanni Pecoraro', 'section': 'Module 6: Best practices', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='733867b9-9ad0-444b-9d63-4933a378a7e0', version=0, score=0.5, payload={'text': "Problem description. How can we connect s3 bucket to MLFLOW?\nSolution: Use boto3 and AWS CLI to store access keys. The access keys are what will be used by boto3 (AWS' Python API tool) to connect with the AWS servers. If there are no Access Keys how can they make sure that they have the right to access this 